In [2]:
import re
import os
import sys
class TreeNode:  
    '''
     x(attr1)
    /\
    x(attr1=3) x(attr1=4)
    '''  
    def __init__(self, attrVal):
        self.partitionAttrVal =attrVal #标记当前node的某个attr的具体值是多少
        self.partitionAttrNum = 0  #标记在当前node下，分割的attr是哪一个
        self.children={}    #保存各种attr的node,leaf node check empty, if not l
        self.ID=None
        self.leafTag=None   #如果是leaf，就赋对应tag的值
        
def preprocess(path):
    print "start reading and preprocessing train data:"
    inputList=[]      #是list of list
    tagSet=set()      #统计tag有多少种，为了后面建立confusion matrix
    
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/balance-scale.train', 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/led.train.new', 'r').readlines()):   
    for index, line in enumerate(open(path, 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/nursery.data.train', 'r').readlines()):   
        
        w=re.split(' |\n',line)  
        del w[-1]      #delete the new line character

        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break

        tagSet.add(w[0])          #记录有多少种tag
        
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        inputList.append(w)     #把处理好的数据加入到总的list of list中            
#     print inputList
#     print tagSet
    print "finish reading and processing train data"
    return inputList,tagSet

def testData(path):
    print "start read and process test data:"
    testList=[]
    
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/balance-scale.test', 'r').readlines()):
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/led.test.new', 'r').readlines()):   
    for index, line in enumerate(open(path, 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/nursery.data.test', 'r').readlines()): 

        w=re.split(' |\n',line)   
        del w[-1]

        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break
            
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        testList.append(w)            
#     print inputList
    print "finish reading and processing test data"
    return testList   

'''
@inputList list of list 注意list of list的下标是从0开始
@rowNums list of integers
'''
def curGini(inputList,rowNums,attrNum):
    tagMap={}
    for rowNum in rowNums:
        curTag=inputList[rowNum][0]
        if curTag in tagMap.keys():
            tagMap[curTag]+=1.0
        else:
            tagMap[curTag]=1.0

    for k,v in tagMap.items():
        print "tag %s has # of rows"%k,
        print v
        
    sum=0.0
    for i in tagMap.keys():
        sum+=pow(tagMap[i]/len(rowNums),2)

    res=1-sum    
    return res  



'''
@inputList list of list
@rowNums list
@availAttrNums list
@root current node
'''
def nextGini(inputList,rowNums,availAttrNums,root,level):
    print
    print "new branch"
    print "level of tree ",
    print level
    print
#     print"root children:",
#     print root.children
#     print "当前节点的分割attribute:",
#     print root.partitionAttrNum
#     if root.par!=None:
#         print "root parent value:",
#         print root.par.partitionAttrVal
#     print "old root ID:",
#     print root.ID
    
    global nodeCount
    root.ID=nodeCount
    nodeCount+=1

#     print "new root ID:",
#     print root.ID
    
    print "row nums: ",
    print rowNums
    print "available attr numbers: ",
    print availAttrNums
    print
    
    """    
    base case
    """
    '''
    case 1, when all the rows in rowNums list have the same label，purity=100%, stop recursion
    '''
    testSet=set()
    testMap={}
    for rowNum in rowNums:
        curList=inputList[rowNum]
        curTag=curList[0]
        if curTag not in testSet:
            testSet.add(curTag)
    if len(testSet)==1:
        root.leafTag=inputList[rowNums[0]][0]
        '''
        print "root tag:",
        print root.leafTag
        '''
        return
    '''
    case 2, when there are no available attributes, use majority voting to decide the label
    '''
    if not availAttrNums:
        for rowNum in rowNums:            
            curList=inputList[rowNum]
            curTag=curList[0]
            if curTag not in testMap:
                testMap[curTag]=1
            else:
                testMap[curTag]+=1
        max=0
        maxKey=''
        
        for k,v in testMap.items():
            if v>max:
                max=v
                maxKey=k
        root.leafTag=maxKey
        print "leaf tag:",
        print root.leafTag
        '''
           print "root tag"
            print root.leafTag
        '''
        
        return 
    
    '''
    non-base case
    '''
    minSum=float("inf")
    chooseAttrNum=-1
    for attrNum in availAttrNums:#availAttrNums是一个string,记录当前branch还有哪些数据，目的是使各个branch互不影响

        curMap={}
        for rowNum in rowNums:
            curList=inputList[rowNum]#获得rowNum行的数据
           
            if curList[attrNum] in curMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
                curMap[curList[attrNum]].append(rowNum)
            else:
                curMap[curList[attrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
                curMap[curList[attrNum]].append(rowNum)

                
        #curMap把某一attribute的所有行放在一起，key attr, val all row numbers
        print
        print "choose attribute %d，its key:label, value:row number map:"%attrNum,
        print curMap
        
        
        #计算某种分割的gini,minSum记录所有分割最小gini,chooseAttrNum记录最小gini对应的attribute number
        total=float(len(rowNums))#当前branch下的总行数
        sum=0.0
        print "calculate gini score of attribute #%d for the next level"%attrNum

        for k,v in curMap.items():
            print "when attribute #%d =%s"%(attrNum,k)
            ratio=len(v)/total
            
            curTerm=ratio*curGini(inputList,v,attrNum)#关键一步
            print "gini score for this term:",
            print curTerm
            sum+=curTerm
            
        print "total gini score for attribute #%d :"%attrNum
        print sum
        if sum<minSum:
            minSum=sum
            chooseAttrNum=attrNum

    
    root.partitionAttrNum=chooseAttrNum#当前node记录下用了哪个attribute分割
    print
    print "attribute with min gini score:",
    print root.partitionAttrNum
    
    #没有gain怎么停止分割？
    
    #重新根据选中的attr建立map
    partitionMap={}
    for rowNum in rowNums:
        curList=inputList[rowNum]#获得rowNum行的数据

        if curList[chooseAttrNum] in partitionMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
        else:
            partitionMap[curList[chooseAttrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
    print
    print "下一层的partition map",
    print partitionMap
    
    
    #生成新的 available attribute list,同时删除用过的attribute
    newAvailAttrNums=list(availAttrNums)
    newAvailAttrNums.remove(chooseAttrNum)
    
    for k,v in partitionMap.items():
        
        newRowNums=v
        newRoot=TreeNode(k)  #生成child 节点
        root.children[k]=newRoot  #把child 节点连接到当前节点上

        nextGini(inputList,newRowNums,newAvailAttrNums,newRoot,level+1) #下一层循环
    
        

if __name__ == "__main__":

    inPath = sys.argv[1]
    outPath=sys.argv[2]
    if os.path.exists(inPath):
        print "train file path is",
        print os.path.basename(inPath)
        print "test file path is",
        print os.path.basename(outPath)
        
    nodeCount=0
    inputList,tagSet=preprocess(inPath)#list of list
    
##检查preprocess 后是否得到所有的行，并且它们的格式都对
#     print "input list is: ",
#     print inputList

    #list记录所有的行数
    rowNums=[i for i in range(len(inputList))]
#     print rowNums
    
    #list记录所有的attribute num
    availAttrNums=[i for i in range(1,len(inputList[0])-1)]
#     print availAttrNums
        
    root=TreeNode(-1)
    level=0   
    
    nextGini(inputList,rowNums,availAttrNums,root,level)
    


    '''
    只是traverse tree,能被用来检查tree的结构
    '''
# # #     """
# # #     pre order traversal
# # #     """
# # #     print
# # #     print "遍历树"
# # #     stack=[]
# # #     stack.append(root)  
# # #     while stack:
# # #         cur=stack.pop()
# # #         print "current ID:"
# # #         print cur.ID        

# # #         if not cur.children:
# # #             print "leaf tag:",
# # #             print cur.leafTag
# # #             continue
# # # #         print "current node's list length:"
# # # #         print len(cur.children)
# # #         for k,v in cur.children.items():
# # # #             print type(k)
# # #             stack.append(v)

    '''
    build confusion matrix
    '''
    
    '''
    print "number of tags:",
    print len(tagSet)
    '''
    
    outPutListList=[]
    for i in range(len(tagSet)):
        tmpList=[0 for j in range(len(tagSet))]
        outPutListList.append(tmpList)
    '''
        for i in outPutListList:
        print i
    '''
        
    
 
    print
    print "test phase"
    trueCount=0
    falseCount=0
    testList=testData(outPath)#list of list
    for test in testList:
    
        stack=[]
        stack.append(root)  
        while stack:
            cur=stack.pop()
#             print "current ID:"
#             print cur.ID    

            testAttrValue=str(test[cur.partitionAttrNum])
#             print "test attr value"
#             print testAttrValue

            if not cur.children:#如果当前的label不在数据集了，就算作第一个branch
                '''
                print "leaf tag:",
                print cur.leafTag
                
                '''
                
                outPutListList[int(test[0])-1][int(cur.leafTag)-1]+=1
                
                if cur.leafTag==test[0]:
                    '''
                    print "true"
                    '''
                    trueCount+=1
                else:
                    '''
                    print "false"
                    '''
                    falseCount+=1
                break
            if testAttrValue in cur.children.keys():
                newNode=cur.children[testAttrValue]
            else:
                newNode=cur.children[cur.children.keys()[0]]
            stack.append(newNode)
            

    #output confusion matrix
    print "confusion matrix:"
    for i in outPutListList:
        print i
        
    '''
    accuracy
    (TP+TN)/(P+N)
    '''
    numeritor=0
    denominator=0
    for i in range(len(outPutListList)):
        numeritor+=outPutListList[i][i]

    for i in range(len(outPutListList)):
        for j in range(len(outPutListList)):
            denominator+=outPutListList[i][j]
    print "accuracy is:",
    print float(numeritor)/float(denominator)
        

start reading and preprocessing train data:
finish reading and processing train data

new branch
level of tree  0

row nums:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196